In [ ]:
county_list = ["新北市", "基隆市", "台北市", "宜蘭縣", "桃園市", 
               "新竹市", "新竹縣", "苗栗縣", "台中市", "彰化縣",
               "南投縣", "雲林縣", "嘉義市", "嘉義縣", "台南市",
               "高雄市", "澎湖縣", "屏東縣", "花蓮縣", "台東縣"]
result = []
n = 0
for county in county_list:
    n += 1
    pref = {
        "county": county,
        "dates": 2,
        "departure_date": "2023-08-01",
        "return_date": "2023-08-02",
        "hotel_level": 4,
        "food_level": 3,
        "viewpoint_level": 5,
        "viewpoint_other_tag": ["藝術展覽","歷史文化"],
        "transportation": 2,
        "food_taste_tag": ["火鍋", "日式料理"],
        "food_price_tag": 2,
        "food_other_tag": ["親子餐廳"],
        "hotel_price_tag": 3000,
        "hotel_like_tag": ["飯店/旅館/酒店"],
        "hotel_other_tag": ["停車場"]
    }

    weekday_name = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]

    # itinerary_df, concat_df = mainrun(kwargs, weekday_name)

    # print(itinerary_df)
    # itinerary_df

    import pandas as pd
    import numpy as np
    # from sqlalchemy import create_engine
    from shapely.geometry import Point
    # from geopy.distance import geodesic
    import warnings
    warnings.filterwarnings("ignore")
    import geopandas as gpd
    from main_function import get_table
    from my_score import calculate_price_score, calculate_price_score_2, calculate_tag_score,calculate_tag_score_2,calculate_price_score_3
    from my_score import calculate_hotel_score_2, calculate_hotel_score # ***

    # 分出群集: 用經緯度 將所有的地點進行分群(clusters)，並畫 map1 以檢視"所有的"clusters分布的狀況。(地圖僅做檢查用 最後要拉掉)
    from sklearn.cluster import DBSCAN
    from sklearn.metrics.pairwise import haversine_distances
    from math import radians
    from random import randint
    import random
    import folium
    from folium.plugins import MarkerCluster




    # 傳入資料
    pref = pref

    # 資料庫
    food_select = f"select * from food_v3 where city = '{pref['county']}'"
    hotel_select = f"select * from hotel_v1 where city = '{pref['county']}'"
    viewpoint_select = f"select * from viewpoint_v3 where city = '{pref['county']}'"

    df_viewpoint = get_table(viewpoint_select)
    df_viewpoint = gpd.GeoDataFrame(df_viewpoint, crs="EPSG:4326", geometry=[Point(lnglat) for lnglat in zip(df_viewpoint['lng'], df_viewpoint['lat'])])

    df_food = get_table(food_select)
    df_food = gpd.GeoDataFrame(df_food, crs="EPSG:4326", geometry=[Point(lnglat) for lnglat in zip(df_food['lng'], df_food['lat'])])

    df_hotel = get_table(hotel_select)
    df_hotel = gpd.GeoDataFrame(df_hotel, crs="EPSG:4326", geometry=[Point(lnglat) for lnglat in zip(df_hotel['lng'], df_hotel['lat'])])



    # 給Hotel分數
    df_hotel['like_score'] = calculate_tag_score_2(df_hotel['keyword'], pref["hotel_like_tag"])
    df_hotel['price_score'] = calculate_hotel_score_2(df_hotel['lower_price'], df_hotel['ceiling_price'],  pref['hotel_price_tag']) # ***
    df_hotel['other_score'] = calculate_tag_score_2(df_hotel['tag'], pref["hotel_other_tag"])/5  # 附加的服務所以權重小一點 # ***

    # 計算Hotel加總分數 
    df_hotel['overall_score'] = df_hotel['like_score'] + df_hotel['price_score'] + df_hotel['other_score']

    # 給Food分數
    df_food['taste_score'] = calculate_tag_score_2(df_food['taste_tag'], pref["food_taste_tag"])
    df_food['price_score'] = calculate_price_score_3(df_food['price_level'], pref['food_price_tag'])
    df_food['other_score'] = calculate_tag_score_2(df_food['other_tag'], pref["food_other_tag"])*10  #屬於重要需求所以權重高一點 # *** 

    # 計算Food加總分數 
    df_food['overall_score'] = df_food['taste_score'] + df_food['price_score'] + df_food['other_score']

    # 給ViewPoint分數
    df_viewpoint['other_score'] = calculate_tag_score_2(df_viewpoint['tag'], pref["viewpoint_other_tag"])

    # 計算ViewPoint加總分數
    df_viewpoint['overall_score'] = df_viewpoint['other_score']

    # df_viewpoint.sort_values(by=['overall_score'], ascending=False).head(5)

    # concat 三張表 for next step
    df_food['source']='food'
    df_viewpoint['source']='viewpoint'
    df_hotel['source']='hotel'
    concat_df = pd.concat([df_food, df_viewpoint, df_hotel])
    # print(concat_df.shape)

    # concat_df.head(5)



    geo_df = concat_df
    print(geo_df.shape)

    # 将经纬度转换为弧度
    geo_df['lat_rad'] = geo_df['lat'].apply(radians)
    geo_df['lng_rad'] = geo_df['lng'].apply(radians)


    # 计算距离矩阵
    haversine_matrix = haversine_distances(
        geo_df[['lat_rad', 'lng_rad']].values,
        geo_df[['lat_rad', 'lng_rad']].values
    ) * 6371000.0  # 6371000.0 is Earth's radius in meters

    # 运行DBSCAN进行聚类
    clus_distance = 2000  # *** 
    dbscan = DBSCAN(eps=clus_distance, min_samples=3, metric='precomputed') # *** 
    geo_df['cluster'] = dbscan.fit_predict(haversine_matrix)

    # 分析聚类结果
    num_clusters = len(set(geo_df['cluster'])) - (1 if -1 in geo_df['cluster'] else 0)
    print(f"Number of clusters: {num_clusters}")

    # 创建地图1：显示所有地点的地图
    m1 = folium.Map(location=[geo_df['lat'].mean(), geo_df['lng'].mean()], zoom_start=12)

    # 添加 MarkerCluster 层
    marker_cluster1 = MarkerCluster().add_to(m1)

    # 在地图上添加地点
    for _, row in geo_df.iterrows():
        folium.Marker(
            location=[row['lat'], row['lng']],
            popup=row['name'],
            icon=None  # 使用默认图标
        ).add_to(marker_cluster1)

    # 保存地图1为HTML文件 (或者 display)
    m1.save(f"{n}_map1_all_clusters_{pref['county']}.html") # ***
    # display(m1)
    cluster_sizes = geo_df['cluster'].value_counts()

    # return num_clusters, cluster_sizes
    result.append({pref['county']: {"distance_parameter": clus_distance, "num_clusters": num_clusters, "cluster_sizes": cluster_sizes.to_dict()}})

In [13]:
# result存成json檔
import json
with open('result.json', 'w', encoding='utf-8', ) as fp:
    json.dump(result, fp)